In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Mon Feb 26 21:03:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install -U spacy


In [ ]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Using the Spacy NER Model for Named Entity Recognition on the Signal-1M Dataset and displaying the output

In [ ]:
!sudo add-apt-repository ppa:alessandro-strada/ppa
!sudo apt-get update
!sudo apt-get install google-drive-ocamlfuse

Repository: 'deb https://ppa.launchpadcontent.net/alessandro-strada/ppa/ubuntu/ jammy main'
Description:
Mount Google Drive on Ubuntu (via FUSE)
More info: https://launchpad.net/~alessandro-strada/+archive/ubuntu/ppa
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/alessandro-strada-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/alessandro-strada-ubuntu-ppa-jammy.list
Adding key to /etc/apt/trusted.gpg.d/alessandro-strada-ubuntu-ppa.gpg with fingerprint 9EA4D6FCA5D37A5D1CA9C09AAD5F235DF639B041
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [713 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy In

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp C:/Users/USER/Documents/University of Alberta/Winter Semester 2024/Knowledge Graphs/Project/sample-1M.jsonl /content/drive/MyDrive/ColabNotebooks/sample-1M.jsonl

cp: target '/content/drive/MyDrive/ColabNotebooks/Project/sample-1M.jsonl' is not a directory


In [ ]:
import spacy
from spacy.tokens import Span
import json
from IPython.core.display import display, HTML

# Load spaCy English model for NER
nlp = spacy.load("en_core_web_sm", disable=["tagger"])

# Define the types of entities you want to visualize
entity_types = ["PERSON", "LOC", "ORG"]

# Path to your dataset file
dataset_path = "/content/drive/MyDrive/first_100k_articles.jsonl"
output_path = "/content/drive/MyDrive/NER_sample-1M.jsonl"

# Define a custom function to generate HTML with underlined entities
def render_entities(doc):
    html = "<div>"
    for ent in doc.ents:
        if ent.label_ in entity_types:
            html += "<u>" + ent.text + "</u>" + f" ({ent.label_})" + " "
        else:
            html += ent.text + " "
    html += "</div>"
    return html

# Open the dataset file
with open(dataset_path, "r", encoding="utf-8") as file, open(output_path, "a", encoding="utf-8") as output_file:
    # Iterate over each line in the dataset file
    for line in file:
        # Parse the JSON line
        data = json.loads(line)

        # Extract text from the data
        text = data["content"]

        # Perform NER
        doc = nlp(text)

        # Render HTML with underlined entities
        html_content = render_entities(doc)
        #display(HTML(html_content))
        # Update the content in the data with marked text
        data["content"] = html_content

        # Write the modified data to the output dataset file
        output_file.write(json.dumps(data) + "\n")


In [ ]:
import json

# Define the file paths
input_file_path = "/content/drive/MyDrive/sample-1M.jsonl"
output_file_path = "/content/drive/MyDrive/first_100k_articles.jsonl"

# Open the input JSONL file and the output JSONL file
with open(input_file_path, "r", encoding="utf-8") as input_file, \
        open(output_file_path, "w", encoding="utf-8") as output_file:

    # Extract the first 100,000 articles
    for _ in range(700000, 1000000):
        line = input_file.readline().strip()  # Read a line from the input file
        if not line:
            break  # End of file reached
        article = json.loads(line)  # Parse the JSON object
        output_file.write(json.dumps(article) + "\n")  # Write the article to the output file


In [ ]:
from IPython.display import display, HTML
import json

# Path to the output dataset file
output_dataset_path = "/content/drive/MyDrive/NER_sample-1M.jsonl"

# Open the output dataset file and display its contents
with open(output_dataset_path, "r", encoding="utf-8") as output_file:
    for line in output_file:
        # Parse the JSON line
        data = json.loads(line)

        # Display the content with underlined texts
        display(HTML(data["content"]))


In [ ]:
#Counting the identified entities grouped by type
import json
import re

def count_entities_by_type(filename):
    entity_counts = {'PERSON': 0, 'ORG': 0, 'LOC': 0}

    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            text = data['content']
            entities = re.findall(r'<u>(.*?)</u>\s*\((\w+)\)', text)
            for entity, entity_type in entities:
                if entity_type in entity_counts:
                    entity_counts[entity_type] += 1

    return entity_counts

filename = "/content/drive/MyDrive/NER_sample-1M.jsonl"
counts = count_entities_by_type(filename)
print("Entity counts by type:")
print(counts)


Entity counts by type:
{'PERSON': 5440707, 'ORG': 6652882, 'LOC': 452827}


In [2]:
import json

# Define the file paths
input_file_path = "/content/drive/MyDrive/NER_sample-1M.jsonl"
output_file_path = "/content/drive/MyDrive/first_100k_articles.jsonl"

# Open the input JSONL file and the output JSONL file
with open(input_file_path, "r", encoding="utf-8") as input_file, \
        open(output_file_path, "w", encoding="utf-8") as output_file:

    # Extract the first 100,000 articles
    for _ in range(0, 4):
        line = input_file.readline().strip()  # Read a line from the input file
        if not line:
            break  # End of file reached
        article = json.loads(line)  # Parse the JSON object
        output_file.write(json.dumps(article) + "\n")  # Write the article to the output file


In [ ]:
from IPython.display import display, HTML
import json

# Path to the output dataset file
output_dataset_path = "/content/drive/MyDrive/NER_sample-1M.jsonl"

# Open the output dataset file and display its contents
with open(output_dataset_path, "r", encoding="utf-8") as output_file:
    for line in output_file:
        # Parse the JSON line
        data = json.loads(line)

        # Display the content with underlined texts
        display(HTML(data["content"]))